In [7]:
# Add execute permissions to binaries
!chmod +x dssp
!chmod +x DAlphaBall.gcc

# Verify permissions
!ls -l dssp
!ls -l DAlphaBall.gcc

-rwxr-xr-x@ 1 satishgaurav  staff  877904 May  4 21:24 dssp
-rwxr-xr-x@ 1 satishgaurav  staff  345824 May  4 21:24 DAlphaBall.gcc


In [8]:
from IPython.display import display, Markdown

In [9]:
####################################
################ BioPython functions
####################################
### Import dependencies
import os
import math
import numpy as np
from collections import defaultdict
from scipy.spatial import cKDTree
from Bio import BiopythonWarning
from Bio.PDB import PDBParser, DSSP, Selection, Polypeptide, PDBIO, Select, Chain, Superimposer
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB.Selection import unfold_entities
from Bio.PDB.Polypeptide import is_aa

# analyze sequence composition of design
def validate_design_sequence(sequence, num_clashes, advanced_settings):
    note_array = []

    # Check if protein contains clashes after relaxation
    if num_clashes > 0:
        note_array.append('Relaxed structure contains clashes.')

    # Check if the sequence contains disallowed amino acids
    if advanced_settings["omit_AAs"]:
        restricted_AAs = advanced_settings["omit_AAs"].split(',')
        for restricted_AA in restricted_AAs:
            if restricted_AA in sequence:
                note_array.append('Contains: '+restricted_AA+'!')

    # Analyze the protein
    analysis = ProteinAnalysis(sequence)

    # Calculate the reduced extinction coefficient per 1% solution
    extinction_coefficient_reduced = analysis.molar_extinction_coefficient()[0]
    molecular_weight = round(analysis.molecular_weight() / 1000, 2)
    extinction_coefficient_reduced_1 = round(extinction_coefficient_reduced / molecular_weight * 0.01, 2)

    # Check if the absorption is high enough
    if extinction_coefficient_reduced_1 <= 2:
        note_array.append(f'Absorption value is {extinction_coefficient_reduced_1}, consider adding tryptophane to design.')

    # Join the notes into a single string
    notes = ' '.join(note_array)

    return notes

# temporary function, calculate RMSD of input PDB and trajectory target
def target_pdb_rmsd(trajectory_pdb, starting_pdb, chain_ids_string):
    # Parse the PDB files
    parser = PDBParser(QUIET=True)
    structure_trajectory = parser.get_structure('trajectory', trajectory_pdb)
    structure_starting = parser.get_structure('starting', starting_pdb)
    
    # Extract chain A from trajectory_pdb
    chain_trajectory = structure_trajectory[0]['A']
    
    # Extract the specified chains from starting_pdb
    chain_ids = chain_ids_string.split(',')
    residues_starting = []
    for chain_id in chain_ids:
        chain_id = chain_id.strip()
        chain = structure_starting[0][chain_id]
        for residue in chain:
            if is_aa(residue, standard=True):
                residues_starting.append(residue)
    
    # Extract residues from chain A in trajectory_pdb
    residues_trajectory = [residue for residue in chain_trajectory if is_aa(residue, standard=True)]
    
    # Ensure that both structures have the same number of residues
    min_length = min(len(residues_starting), len(residues_trajectory))
    residues_starting = residues_starting[:min_length]
    residues_trajectory = residues_trajectory[:min_length]
    
    # Collect CA atoms from the two sets of residues
    atoms_starting = [residue['CA'] for residue in residues_starting if 'CA' in residue]
    atoms_trajectory = [residue['CA'] for residue in residues_trajectory if 'CA' in residue]
    
    # Calculate RMSD using structural alignment
    sup = Superimposer()
    sup.set_atoms(atoms_starting, atoms_trajectory)
    rmsd = sup.rms
    
    return round(rmsd, 2)

# detect C alpha clashes for deformed trajectories
def calculate_clash_score(pdb_file, threshold=2.4, only_ca=False):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)

    atoms = []
    atom_info = []  # Detailed atom info for debugging and processing

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    if atom.element == 'H':  # Skip hydrogen atoms
                        continue
                    if only_ca and atom.get_name() != 'CA':
                        continue
                    atoms.append(atom.coord)
                    atom_info.append((chain.id, residue.id[1], atom.get_name(), atom.coord))

    tree = cKDTree(atoms)
    pairs = tree.query_pairs(threshold)

    valid_pairs = set()
    for (i, j) in pairs:
        chain_i, res_i, name_i, coord_i = atom_info[i]
        chain_j, res_j, name_j, coord_j = atom_info[j]

        # Exclude clashes within the same residue
        if chain_i == chain_j and res_i == res_j:
            continue

        # Exclude directly sequential residues in the same chain for all atoms
        if chain_i == chain_j and abs(res_i - res_j) == 1:
            continue

        # If calculating sidechain clashes, only consider clashes between different chains
        if not only_ca and chain_i == chain_j:
            continue

        valid_pairs.add((i, j))

    return len(valid_pairs)

three_to_one_map = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

# identify interacting residues at the binder interface
def hotspot_residues(trajectory_pdb, binder_chain="B", atom_distance_cutoff=4.0):
    # Parse the PDB file
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", trajectory_pdb)

    # Get the specified chain
    binder_atoms = Selection.unfold_entities(structure[0][binder_chain], 'A')
    binder_coords = np.array([atom.coord for atom in binder_atoms])

    # Get atoms and coords for the target chain
    target_atoms = Selection.unfold_entities(structure[0]['A'], 'A')
    target_coords = np.array([atom.coord for atom in target_atoms])

    # Build KD trees for both chains
    binder_tree = cKDTree(binder_coords)
    target_tree = cKDTree(target_coords)

    # Prepare to collect interacting residues
    interacting_residues = {}

    # Query the tree for pairs of atoms within the distance cutoff
    pairs = binder_tree.query_ball_tree(target_tree, atom_distance_cutoff)

    # Process each binder atom's interactions
    for binder_idx, close_indices in enumerate(pairs):
        binder_residue = binder_atoms[binder_idx].get_parent()
        binder_resname = binder_residue.get_resname()

        # Convert three-letter code to single-letter code using the manual dictionary
        if binder_resname in three_to_one_map:
            aa_single_letter = three_to_one_map[binder_resname]
            for close_idx in close_indices:
                target_residue = target_atoms[close_idx].get_parent()
                interacting_residues[binder_residue.id[1]] = aa_single_letter

    return interacting_residues

# calculate secondary structure percentage of design
def calc_ss_percentage(pdb_file, advanced_settings, chain_id="B", atom_distance_cutoff=4.0):
    # Parse the structure
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]  # Consider only the first model in the structure

    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')

    # Prepare to count residues
    ss_counts = defaultdict(int)
    ss_interface_counts = defaultdict(int)
    plddts_interface = []
    plddts_ss = []

    # Get chain and interacting residues once
    chain = model[chain_id]
    interacting_residues = set(hotspot_residues(pdb_file, chain_id, atom_distance_cutoff).keys())

    for residue in chain:
        residue_id = residue.id[1]
        if (chain_id, residue_id) in dssp:
            ss = dssp[(chain_id, residue_id)][2]  # Get the secondary structure
            ss_type = 'loop'
            if ss in ['H', 'G', 'I']:
                ss_type = 'helix'
            elif ss == 'E':
                ss_type = 'sheet'

            ss_counts[ss_type] += 1

            if ss_type != 'loop':
                # calculate secondary structure normalised pLDDT
                avg_plddt_ss = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_ss.append(avg_plddt_ss)

            if residue_id in interacting_residues:
                ss_interface_counts[ss_type] += 1

                # calculate interface pLDDT
                avg_plddt_residue = sum(atom.bfactor for atom in residue) / len(residue)
                plddts_interface.append(avg_plddt_residue)

    # Calculate percentages
    total_residues = sum(ss_counts.values())
    total_interface_residues = sum(ss_interface_counts.values())

    percentages = calculate_percentages(total_residues, ss_counts['helix'], ss_counts['sheet'])
    interface_percentages = calculate_percentages(total_interface_residues, ss_interface_counts['helix'], ss_interface_counts['sheet'])

    i_plddt = round(sum(plddts_interface) / len(plddts_interface) / 100, 2) if plddts_interface else 0
    ss_plddt = round(sum(plddts_ss) / len(plddts_ss) / 100, 2) if plddts_ss else 0

    return (*percentages, *interface_percentages, i_plddt, ss_plddt)

def calculate_percentages(total, helix, sheet):
    helix_percentage = round((helix / total) * 100,2) if total > 0 else 0
    sheet_percentage = round((sheet / total) * 100,2) if total > 0 else 0
    loop_percentage = round(((total - helix - sheet) / total) * 100,2) if total > 0 else 0

    return helix_percentage, sheet_percentage, loop_percentage

In [10]:
advanced_settings = {
    "dssp_path": "/usr/local/bin/mkdssp",
    "omit_AAs": "C",
}
pdb_file = "./../target/7z14.pdb"
calc_ss_percentage(pdb_file=pdb_file, advanced_settings=advanced_settings)

/opt/miniconda3/envs/alphafold/lib/python3.10/site-packages/Bio/PDB/DSSP.py:199: UserWarning: Links for pdbx_branch_scheme:atom_site:1 are incomplete
  There are 27 items in pdbx_branch_scheme that don't have matching parent items in atom_site
Resulting mmCIF file is not valid!

  warnings.warn(err)


(35.92, 20.64, 43.43, 38.78, 2.04, 59.18, 0.99, 1.0)

In [11]:
import pandas as pd

AA_PROPERTIES = {
        "ALA": "nonpolar",
        "ARG": "positive",
        "ASN": "polar",
        "ASP": "negative",
        "CYS": "polar",
        "GLN": "polar",
        "GLU": "negative",
        "GLY": "nonpolar",
        "HIS": "positive",
        "ILE": "nonpolar",
        "LEU": "nonpolar",
        "LYS": "positive",
        "MET": "nonpolar",
        "PHE": "nonpolar",
        "PRO": "nonpolar",
        "SER": "polar",
        "THR": "polar",
        "TRP": "nonpolar",
        "TYR": "polar",
        "VAL": "nonpolar",
        "SEC": "polar"
    }

def create_structure_df(pdb_file, advanced_settings, chain_id=None, atom_distance_cutoff=4.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdb_file)
    model = structure[0]
    
    # Calculate DSSP for the model
    dssp = DSSP(model, pdb_file, dssp='mkdssp')
    
    # Prepare data for DataFrame
    data = []
    
    # Process either specific chain or all chains
    chains = [model[chain_id]] if chain_id else model.get_chains()
    
    for chain in chains:
        current_chain_id = chain.id
        # Get interacting residues if chain_id is specified
        interacting_residues = set()
        if current_chain_id:
            interacting_residues = set(hotspot_residues(pdb_file, current_chain_id, atom_distance_cutoff).keys())
        
        for residue in chain:
            residue_id = residue.id[1]
            res_name = residue.get_resname()
            
            if (current_chain_id, residue_id) in dssp:
                dssp_data = dssp[(current_chain_id, residue_id)]
                ss = dssp_data[2]
                acc = dssp_data[3]
                phi = dssp_data[4]
                psi = dssp_data[5]
                
                ss_type = 'loop'
                if ss in ['H', 'G', 'I']:
                    ss_type = 'helix'
                elif ss == 'E':
                    ss_type = 'sheet'
                    
                avg_plddt = sum(atom.bfactor for atom in residue) / len(residue) / 100
                
                data.append({
                    'chain_id': current_chain_id,
                    'residue_id': residue_id,
                    'residue_name': res_name,
                    'property': AA_PROPERTIES.get(res_name, 'unknown'),
                    'ss_type': ss_type,
                    'ss_code': ss,
                    'accessibility': acc,
                    'phi': phi,
                    'psi': psi,
                    'plddt': round(avg_plddt, 2),
                    'is_interface': residue_id in interacting_residues if chain_id else None
                })
    
    return pd.DataFrame(data)


# Example usage:
pdb_file = "./../target/7z14.pdb"
df = create_structure_df(pdb_file, advanced_settings)
df.head()

/opt/miniconda3/envs/alphafold/lib/python3.10/site-packages/Bio/PDB/DSSP.py:199: UserWarning: Links for pdbx_branch_scheme:atom_site:1 are incomplete
  There are 27 items in pdbx_branch_scheme that don't have matching parent items in atom_site
Resulting mmCIF file is not valid!

  warnings.warn(err)


,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,SER,polar,loop,-,0.030769,360.0,138.1,0.93,None
1,A,2,GLU,negative,helix,H,0.520619,-67.1,-35.7,0.95,None
2,A,3,HIS,positive,helix,H,0.255435,-63.7,-43.7,0.92,None
3,A,4,GLU,negative,helix,H,0.072165,-69.2,-35.5,0.93,None
4,A,5,THR,polar,helix,H,0.154930,-62.5,-44.7,0.94,None


In [12]:
df.tail(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
1903,G,51,HIS,positive,loop,-,0.554348,-133.4,52.6,1.27,None
1904,G,52,CYS,polar,loop,-,0.555556,-54.1,112.5,1.32,None
1905,G,53,CYS,polar,loop,B,0.096296,-130.6,134.7,1.35,None
1906,G,54,ARG,positive,loop,-,0.770161,-93.3,-16.8,1.37,None
1907,G,55,THR,polar,loop,S,0.690141,-79.8,-167.8,1.37,None
1908,G,56,ASP,negative,loop,S,0.539877,-112.8,137.7,1.38,None
1909,G,57,LYS,positive,loop,S,0.434146,56.5,41.2,1.35,None
1910,G,58,CYS,polar,loop,-,0.192593,-104.5,-9.1,1.34,None
1911,G,59,ASN,polar,loop,-,0.000000,-95.4,27.2,1.28,None
1912,G,60,ASN,polar,loop,-,0.700637,-68.5,360.0,1.27,None


In [13]:
df.head(10)

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
0,A,1,SER,polar,loop,-,0.030769,360.0,138.1,0.93,None
1,A,2,GLU,negative,helix,H,0.520619,-67.1,-35.7,0.95,None
2,A,3,HIS,positive,helix,H,0.255435,-63.7,-43.7,0.92,None
3,A,4,GLU,negative,helix,H,0.072165,-69.2,-35.5,0.93,None
4,A,5,THR,polar,helix,H,0.154930,-62.5,-44.7,0.94,None
5,A,6,ARG,positive,helix,H,0.439516,-62.4,-41.3,0.93,None
6,A,7,LEU,nonpolar,helix,H,0.006098,-64.1,-43.2,0.89,None
7,A,8,VAL,nonpolar,helix,H,0.063380,-64.8,-41.3,0.88,None
8,A,9,ALA,nonpolar,helix,H,0.566038,-63.3,-37.8,0.92,None
9,A,10,ASN,polar,helix,H,0.439490,-70.4,-43.1,0.92,None


In [19]:
df_target = df[df['chain_id'] == 'F']
df_target = df_target.reset_index(drop=True)
df_target.index = range(1, len(df_target) + 1)

In [20]:
df_target.to_csv('7z14_chain_F_secondary_structure_information.csv', index=False)

In [21]:
df_target[df_target['property'] == 'nonpolar']

,chain_id,residue_id,residue_name,property,ss_type,ss_code,accessibility,phi,psi,plddt,is_interface
1,F,1,MET,nonpolar,loop,-,0.308511,360.0,128.5,1.36,None
2,F,2,ILE,nonpolar,sheet,E,0.301775,-109.3,125.0,1.31,None
11,F,11,PRO,nonpolar,loop,P,0.647059,-58.9,144.5,1.13,None
12,F,12,PRO,nonpolar,loop,P,0.669118,-60.5,141.8,1.16,None
27,F,27,TRP,nonpolar,sheet,E,0.105727,-156.8,163.6,1.02,None
32,F,32,GLY,nonpolar,loop,-,0.142857,-156.1,-151.9,0.97,None
34,F,34,ILE,nonpolar,sheet,E,0.047337,-117.1,131.3,1.02,None
35,F,35,ILE,nonpolar,sheet,E,0.159763,-115.0,125.1,1.05,None
38,F,38,GLY,nonpolar,sheet,E,0.059524,178.1,-168.1,1.21,None
40,F,40,GLY,nonpolar,loop,S,0.142857,65.2,-171.1,1.28,None


In [22]:
display(Markdown(df_target[df_target['property'] == 'nonpolar'].sort_values('accessibility', ascending=False).to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 46 | F          |           46 | PRO            | nonpolar   | loop      | T         |       0.882353  |  -60.5 |  143.3 |    1.06 |                |
| 47 | F          |           47 | GLY            | nonpolar   | loop      | T         |       0.761905  |   93.2 |   10.7 |    1.05 |                |
| 12 | F          |           12 | PRO            | nonpolar   | loop      | P         |       0.669118  |  -60.5 |  141.8 |    1.16 |                |
| 11 | F          |           11 | PRO            | nonpolar   | loop      | P         |       0.647059  |  -58.9 |  144.5 |    1.13 |                |
| 42 | F          |           42 | PRO            | nonpolar   | loop      | -         |       0.588235  |  -86.6 | -177.9 |    1.23 |                |
| 44 | F          |           44 | VAL            | nonpolar   | loop      | -         |       0.352113  | -130.8 |  159.6 |    1.12 |                |
| 50 | F          |           50 | LEU            | nonpolar   | sheet     | E         |       0.335366  | -136.3 |  153.4 |    1.17 |                |
|  1 | F          |            1 | MET            | nonpolar   | loop      | -         |       0.308511  |  360   |  128.5 |    1.36 |                |
|  2 | F          |            2 | ILE            | nonpolar   | sheet     | E         |       0.301775  | -109.3 |  125   |    1.31 |                |
| 35 | F          |           35 | ILE            | nonpolar   | sheet     | E         |       0.159763  | -115   |  125.1 |    1.05 |                |
| 32 | F          |           32 | GLY            | nonpolar   | loop      | -         |       0.142857  | -156.1 | -151.9 |    0.97 |                |
| 40 | F          |           40 | GLY            | nonpolar   | loop      | S         |       0.142857  |   65.2 | -171.1 |    1.28 |                |
| 48 | F          |           48 | ILE            | nonpolar   | loop      | -         |       0.112426  | -119.7 |  130.6 |    1.06 |                |
| 27 | F          |           27 | TRP            | nonpolar   | sheet     | E         |       0.105727  | -156.8 |  163.6 |    1.02 |                |
| 38 | F          |           38 | GLY            | nonpolar   | sheet     | E         |       0.0595238 |  178.1 | -168.1 |    1.21 |                |
| 34 | F          |           34 | ILE            | nonpolar   | sheet     | E         |       0.0473373 | -117.1 |  131.3 |    1.02 |                |

In [23]:
display(Markdown(df_target.sort_values('accessibility', ascending=False).to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
| 43 | F          |           43 | LYS            | positive   | loop      | -         |       0.97561   |  -74.4 |  149.3 |    1.2  |                |
| 18 | F          |           18 | SER            | polar      | loop      | S         |       0.892308  |  -86.3 |  -11.9 |    1.42 |                |
| 46 | F          |           46 | PRO            | nonpolar   | loop      | T         |       0.882353  |  -60.5 |  143.3 |    1.06 |                |
| 16 | F          |           16 | THR            | polar      | sheet     | E         |       0.838028  |  -97.4 |  138   |    1.37 |                |
| 47 | F          |           47 | GLY            | nonpolar   | loop      | T         |       0.761905  |   93.2 |   10.7 |    1.05 |                |
| 60 | F          |           60 | ASN            | polar      | loop      | -         |       0.745223  |  -59.9 |  360   |    1.25 |                |
| 54 | F          |           54 | ARG            | positive   | loop      | -         |       0.741935  | -121.3 |  -18.5 |    1.32 |                |
| 49 | F          |           49 | LYS            | positive   | sheet     | E         |       0.731707  |  -91.2 |  141.1 |    1.11 |                |
| 12 | F          |           12 | PRO            | nonpolar   | loop      | P         |       0.669118  |  -60.5 |  141.8 |    1.16 |                |
| 20 | F          |           20 | THR            | polar      | loop      | S         |       0.669014  |  -92.3 |    5.5 |    1.37 |                |
| 55 | F          |           55 | THR            | polar      | loop      | S         |       0.647887  |  -96.3 |  175.4 |    1.32 |                |
| 11 | F          |           11 | PRO            | nonpolar   | loop      | P         |       0.647059  |  -58.9 |  144.5 |    1.13 |                |
| 42 | F          |           42 | PRO            | nonpolar   | loop      | -         |       0.588235  |  -86.6 | -177.9 |    1.23 |                |
| 19 | F          |           19 | GLU            | negative   | loop      | -         |       0.572165  |  -78.3 |  154   |    1.4  |                |
| 51 | F          |           51 | HIS            | positive   | loop      | -         |       0.559783  | -133.8 |   53.2 |    1.25 |                |
| 52 | F          |           52 | CYS            | polar      | loop      | -         |       0.555556  |  -40.9 |  132.5 |    1.31 |                |
| 57 | F          |           57 | LYS            | positive   | loop      | S         |       0.546341  |   55.5 |   63.6 |    1.3  |                |
| 56 | F          |           56 | ASP            | negative   | loop      | S         |       0.546012  |  -81.6 |  132.9 |    1.33 |                |
| 13 | F          |           13 | THR            | polar      | loop      | -         |       0.514085  | -147.2 |  135.7 |    1.2  |                |
| 14 | F          |           14 | THR            | polar      | sheet     | E         |       0.471831  | -117.4 |  158.9 |    1.24 |                |
|  9 | F          |            9 | SER            | polar      | loop      | T         |       0.369231  | -110.5 |   39.9 |    1.06 |                |
| 28 | F          |           28 | ARG            | positive   | sheet     | E         |       0.362903  | -119   |  138.9 |    1.02 |                |
|  5 | F          |            5 | ASN            | polar      | loop      | -         |       0.356688  | -115.1 |   13.9 |    1.19 |                |
| 44 | F          |           44 | VAL            | nonpolar   | loop      | -         |       0.352113  | -130.8 |  159.6 |    1.12 |                |
| 15 | F          |           15 | LYS            | positive   | sheet     | E         |       0.35122   | -145.6 |  157   |    1.28 |                |
| 33 | F          |           33 | THR            | polar      | sheet     | E         |       0.34507   | -112.8 |  139.3 |    1.01 |                |
| 50 | F          |           50 | LEU            | nonpolar   | sheet     | E         |       0.335366  | -136.3 |  153.4 |    1.17 |                |
| 31 | F          |           31 | ARG            | positive   | loop      | S         |       0.314516  |  -99.7 |  -56.6 |    0.94 |                |
| 45 | F          |           45 | LYS            | positive   | loop      | -         |       0.312195  |  -68.9 |  155.8 |    1.06 |                |
|  1 | F          |            1 | MET            | nonpolar   | loop      | -         |       0.308511  |  360   |  128.5 |    1.36 |                |
|  7 | F          |            7 | GLN            | polar      | loop      | B         |       0.308081  | -132.8 |  137   |    1.02 |                |
|  2 | F          |            2 | ILE            | nonpolar   | sheet     | E         |       0.301775  | -109.3 |  125   |    1.31 |                |
| 36 | F          |           36 | GLU            | negative   | sheet     | E         |       0.298969  | -119.1 |  133.3 |    1.1  |                |
| 41 | F          |           41 | CYS            | polar      | loop      | -         |       0.281481  | -157.3 |   98.2 |    1.27 |                |
| 24 | F          |           24 | LYS            | positive   | sheet     | E         |       0.243902  | -119.6 |  122.3 |    1.12 |                |
| 58 | F          |           58 | CYS            | polar      | loop      | -         |       0.237037  | -123.3 |   -0.5 |    1.31 |                |
| 17 | F          |           17 | CYS            | polar      | loop      | -         |       0.222222  | -151.5 |  157.7 |    1.4  |                |
| 26 | F          |           26 | THR            | polar      | sheet     | E         |       0.21831   | -127.8 |  132.1 |    1.06 |                |
| 10 | F          |           10 | GLN            | polar      | loop      | P         |       0.181818  |  -73.4 |  166.8 |    1.07 |                |
| 53 | F          |           53 | CYS            | polar      | loop      | B         |       0.177778  | -147   |  161.1 |    1.32 |                |
| 39 | F          |           39 | CYS            | polar      | sheet     | E         |       0.177778  |  -94.2 |  163.2 |    1.29 |                |
|  4 | F          |            4 | TYR            | polar      | sheet     | E         |       0.166667  |  -70.7 |  138.2 |    1.21 |                |
| 35 | F          |           35 | ILE            | nonpolar   | sheet     | E         |       0.159763  | -115   |  125.1 |    1.05 |                |
| 30 | F          |           30 | HIS            | positive   | loop      | S         |       0.157609  |  -64.2 |  -18.9 |    0.94 |                |
| 32 | F          |           32 | GLY            | nonpolar   | loop      | -         |       0.142857  | -156.1 | -151.9 |    0.97 |                |
| 40 | F          |           40 | GLY            | nonpolar   | loop      | S         |       0.142857  |   65.2 | -171.1 |    1.28 |                |
| 21 | F          |           21 | SER            | polar      | loop      | -         |       0.130769  | -125   |  148.9 |    1.34 |                |
| 48 | F          |           48 | ILE            | nonpolar   | loop      | -         |       0.112426  | -119.7 |  130.6 |    1.06 |                |
| 23 | F          |           23 | TYR            | polar      | sheet     | E         |       0.108108  | -123   |  153.4 |    1.17 |                |
| 27 | F          |           27 | TRP            | nonpolar   | sheet     | E         |       0.105727  | -156.8 |  163.6 |    1.02 |                |
| 37 | F          |           37 | ARG            | positive   | sheet     | E         |       0.0887097 | -127.1 |  146.6 |    1.13 |                |
|  6 | F          |            6 | GLN            | polar      | loop      | -         |       0.0757576 |  -69.7 |  177.8 |    1.12 |                |
| 25 | F          |           25 | LYS            | positive   | sheet     | E         |       0.0682927 | -112.6 |  124.1 |    1.07 |                |
| 38 | F          |           38 | GLY            | nonpolar   | sheet     | E         |       0.0595238 |  178.1 | -168.1 |    1.21 |                |
| 34 | F          |           34 | ILE            | nonpolar   | sheet     | E         |       0.0473373 | -117.1 |  131.3 |    1.02 |                |
| 29 | F          |           29 | ASP            | negative   | loop      | -         |       0.0429448 | -116.9 | -169.3 |    0.98 |                |
| 22 | F          |           22 | CYS            | polar      | sheet     | E         |       0         | -141.6 |  134   |    1.28 |                |
|  8 | F          |            8 | SER            | polar      | loop      | T         |       0         |   60.7 | -131.5 |    1.04 |                |
|  3 | F          |            3 | CYS            | polar      | sheet     | E         |       0         | -150.9 |  162.2 |    1.28 |                |
| 59 | F          |           59 | ASN            | polar      | loop      | -         |       0         |  -99   |   34.3 |    1.26 |                |

In [24]:
display(Markdown(df_target.to_markdown()))

|    | chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |    phi |    psi |   plddt | is_interface   |
|---:|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------:|-------:|--------:|:---------------|
|  1 | F          |            1 | MET            | nonpolar   | loop      | -         |       0.308511  |  360   |  128.5 |    1.36 |                |
|  2 | F          |            2 | ILE            | nonpolar   | sheet     | E         |       0.301775  | -109.3 |  125   |    1.31 |                |
|  3 | F          |            3 | CYS            | polar      | sheet     | E         |       0         | -150.9 |  162.2 |    1.28 |                |
|  4 | F          |            4 | TYR            | polar      | sheet     | E         |       0.166667  |  -70.7 |  138.2 |    1.21 |                |
|  5 | F          |            5 | ASN            | polar      | loop      | -         |       0.356688  | -115.1 |   13.9 |    1.19 |                |
|  6 | F          |            6 | GLN            | polar      | loop      | -         |       0.0757576 |  -69.7 |  177.8 |    1.12 |                |
|  7 | F          |            7 | GLN            | polar      | loop      | B         |       0.308081  | -132.8 |  137   |    1.02 |                |
|  8 | F          |            8 | SER            | polar      | loop      | T         |       0         |   60.7 | -131.5 |    1.04 |                |
|  9 | F          |            9 | SER            | polar      | loop      | T         |       0.369231  | -110.5 |   39.9 |    1.06 |                |
| 10 | F          |           10 | GLN            | polar      | loop      | P         |       0.181818  |  -73.4 |  166.8 |    1.07 |                |
| 11 | F          |           11 | PRO            | nonpolar   | loop      | P         |       0.647059  |  -58.9 |  144.5 |    1.13 |                |
| 12 | F          |           12 | PRO            | nonpolar   | loop      | P         |       0.669118  |  -60.5 |  141.8 |    1.16 |                |
| 13 | F          |           13 | THR            | polar      | loop      | -         |       0.514085  | -147.2 |  135.7 |    1.2  |                |
| 14 | F          |           14 | THR            | polar      | sheet     | E         |       0.471831  | -117.4 |  158.9 |    1.24 |                |
| 15 | F          |           15 | LYS            | positive   | sheet     | E         |       0.35122   | -145.6 |  157   |    1.28 |                |
| 16 | F          |           16 | THR            | polar      | sheet     | E         |       0.838028  |  -97.4 |  138   |    1.37 |                |
| 17 | F          |           17 | CYS            | polar      | loop      | -         |       0.222222  | -151.5 |  157.7 |    1.4  |                |
| 18 | F          |           18 | SER            | polar      | loop      | S         |       0.892308  |  -86.3 |  -11.9 |    1.42 |                |
| 19 | F          |           19 | GLU            | negative   | loop      | -         |       0.572165  |  -78.3 |  154   |    1.4  |                |
| 20 | F          |           20 | THR            | polar      | loop      | S         |       0.669014  |  -92.3 |    5.5 |    1.37 |                |
| 21 | F          |           21 | SER            | polar      | loop      | -         |       0.130769  | -125   |  148.9 |    1.34 |                |
| 22 | F          |           22 | CYS            | polar      | sheet     | E         |       0         | -141.6 |  134   |    1.28 |                |
| 23 | F          |           23 | TYR            | polar      | sheet     | E         |       0.108108  | -123   |  153.4 |    1.17 |                |
| 24 | F          |           24 | LYS            | positive   | sheet     | E         |       0.243902  | -119.6 |  122.3 |    1.12 |                |
| 25 | F          |           25 | LYS            | positive   | sheet     | E         |       0.0682927 | -112.6 |  124.1 |    1.07 |                |
| 26 | F          |           26 | THR            | polar      | sheet     | E         |       0.21831   | -127.8 |  132.1 |    1.06 |                |
| 27 | F          |           27 | TRP            | nonpolar   | sheet     | E         |       0.105727  | -156.8 |  163.6 |    1.02 |                |
| 28 | F          |           28 | ARG            | positive   | sheet     | E         |       0.362903  | -119   |  138.9 |    1.02 |                |
| 29 | F          |           29 | ASP            | negative   | loop      | -         |       0.0429448 | -116.9 | -169.3 |    0.98 |                |
| 30 | F          |           30 | HIS            | positive   | loop      | S         |       0.157609  |  -64.2 |  -18.9 |    0.94 |                |
| 31 | F          |           31 | ARG            | positive   | loop      | S         |       0.314516  |  -99.7 |  -56.6 |    0.94 |                |
| 32 | F          |           32 | GLY            | nonpolar   | loop      | -         |       0.142857  | -156.1 | -151.9 |    0.97 |                |
| 33 | F          |           33 | THR            | polar      | sheet     | E         |       0.34507   | -112.8 |  139.3 |    1.01 |                |
| 34 | F          |           34 | ILE            | nonpolar   | sheet     | E         |       0.0473373 | -117.1 |  131.3 |    1.02 |                |
| 35 | F          |           35 | ILE            | nonpolar   | sheet     | E         |       0.159763  | -115   |  125.1 |    1.05 |                |
| 36 | F          |           36 | GLU            | negative   | sheet     | E         |       0.298969  | -119.1 |  133.3 |    1.1  |                |
| 37 | F          |           37 | ARG            | positive   | sheet     | E         |       0.0887097 | -127.1 |  146.6 |    1.13 |                |
| 38 | F          |           38 | GLY            | nonpolar   | sheet     | E         |       0.0595238 |  178.1 | -168.1 |    1.21 |                |
| 39 | F          |           39 | CYS            | polar      | sheet     | E         |       0.177778  |  -94.2 |  163.2 |    1.29 |                |
| 40 | F          |           40 | GLY            | nonpolar   | loop      | S         |       0.142857  |   65.2 | -171.1 |    1.28 |                |
| 41 | F          |           41 | CYS            | polar      | loop      | -         |       0.281481  | -157.3 |   98.2 |    1.27 |                |
| 42 | F          |           42 | PRO            | nonpolar   | loop      | -         |       0.588235  |  -86.6 | -177.9 |    1.23 |                |
| 43 | F          |           43 | LYS            | positive   | loop      | -         |       0.97561   |  -74.4 |  149.3 |    1.2  |                |
| 44 | F          |           44 | VAL            | nonpolar   | loop      | -         |       0.352113  | -130.8 |  159.6 |    1.12 |                |
| 45 | F          |           45 | LYS            | positive   | loop      | -         |       0.312195  |  -68.9 |  155.8 |    1.06 |                |
| 46 | F          |           46 | PRO            | nonpolar   | loop      | T         |       0.882353  |  -60.5 |  143.3 |    1.06 |                |
| 47 | F          |           47 | GLY            | nonpolar   | loop      | T         |       0.761905  |   93.2 |   10.7 |    1.05 |                |
| 48 | F          |           48 | ILE            | nonpolar   | loop      | -         |       0.112426  | -119.7 |  130.6 |    1.06 |                |
| 49 | F          |           49 | LYS            | positive   | sheet     | E         |       0.731707  |  -91.2 |  141.1 |    1.11 |                |
| 50 | F          |           50 | LEU            | nonpolar   | sheet     | E         |       0.335366  | -136.3 |  153.4 |    1.17 |                |
| 51 | F          |           51 | HIS            | positive   | loop      | -         |       0.559783  | -133.8 |   53.2 |    1.25 |                |
| 52 | F          |           52 | CYS            | polar      | loop      | -         |       0.555556  |  -40.9 |  132.5 |    1.31 |                |
| 53 | F          |           53 | CYS            | polar      | loop      | B         |       0.177778  | -147   |  161.1 |    1.32 |                |
| 54 | F          |           54 | ARG            | positive   | loop      | -         |       0.741935  | -121.3 |  -18.5 |    1.32 |                |
| 55 | F          |           55 | THR            | polar      | loop      | S         |       0.647887  |  -96.3 |  175.4 |    1.32 |                |
| 56 | F          |           56 | ASP            | negative   | loop      | S         |       0.546012  |  -81.6 |  132.9 |    1.33 |                |
| 57 | F          |           57 | LYS            | positive   | loop      | S         |       0.546341  |   55.5 |   63.6 |    1.3  |                |
| 58 | F          |           58 | CYS            | polar      | loop      | -         |       0.237037  | -123.3 |   -0.5 |    1.31 |                |
| 59 | F          |           59 | ASN            | polar      | loop      | -         |       0         |  -99   |   34.3 |    1.26 |                |
| 60 | F          |           60 | ASN            | polar      | loop      | -         |       0.745223  |  -59.9 |  360   |    1.25 |                |

In [25]:
from Bio.PDB import PDBParser
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Data import IUPACData
from Bio.SeqUtils import ProtParamData

# 3-letter to 1-letter code map
three_to_one = IUPACData.protein_letters_3to1

# Load PDB
pdb_file = "./../target/7z14.pdb"
parser = PDBParser()
structure = parser.get_structure("7z14", pdb_file)

# Extract sequence and map residues
residue_list = []
sequence = ""

for chain in structure[0]:  # First model
    if chain.id == 'F':
        for residue in chain:
            resname = residue.get_resname().strip()
            try:
                one_letter = three_to_one[resname.capitalize()]
                sequence += one_letter
                residue_list.append(residue)
            except KeyError:
                continue  # Skip non-standard residues

print("Sequence:", sequence)
print("Length:", len(sequence))

# Hydropathy analysis with window
window = 9
pa = ProteinAnalysis(sequence)
hydropathy_values = pa.protein_scale(param_dict=ProtParamData.kd, window=window)

# Align hydropathy values to the residue list
offset = window // 2  # Values are centered in the window

for i, residue in enumerate(residue_list):
    resname = residue.get_resname()
    resnum = residue.get_id()[1]
    if offset <= i < len(residue_list) - offset:
        hydro = hydropathy_values[i - offset]
        print(f"Residue {resname} {resnum:>3}: Hydropathy = {hydro:.2f}")
    else:
        print(f"Residue {resname} {resnum:>3}: Hydropathy = NA")


Sequence: MICYNQQSSQPPTTKTCSETSCYKKTWRDHRGTIIERGCGCPKVKPGIKLHCCRTDKCNN
Length: 60
Residue MET   1: Hydropathy = NA
Residue ILE   2: Hydropathy = NA
Residue CYS   3: Hydropathy = NA
Residue TYR   4: Hydropathy = NA
Residue ASN   5: Hydropathy = -0.50
Residue GLN   6: Hydropathy = -1.10
Residue GLN   7: Hydropathy = -1.78
Residue SER   8: Hydropathy = -2.23
Residue SER   9: Hydropathy = -2.17
Residue GLN  10: Hydropathy = -1.86
Residue PRO  11: Hydropathy = -1.90
Residue PRO  12: Hydropathy = -1.59
Residue THR  13: Hydropathy = -1.22
Residue THR  14: Hydropathy = -1.22
Residue LYS  15: Hydropathy = -1.22
Residue THR  16: Hydropathy = -1.12
Residue CYS  17: Hydropathy = -1.03
Residue SER  18: Hydropathy = -0.68
Residue GLU  19: Hydropathy = -0.74
Residue THR  20: Hydropathy = -0.74
Residue SER  21: Hydropathy = -1.10
Residue CYS  22: Hydropathy = -1.46
Residue TYR  23: Hydropathy = -1.47
Residue LYS  24: Hydropathy = -1.58
Residue LYS  25: Hydropathy = -1.89
Residue THR  26: Hydropathy = 

/opt/miniconda3/envs/alphafold/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17117.
  warnings.warn(


In [26]:
import pandas as pd
from Bio.PDB import PDBParser
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.Data import IUPACData
from Bio.SeqUtils import ProtParamData

def calculate_hydropathy_df(pdb_file, chain_id='F', window=9):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    
    three_to_one = IUPACData.protein_letters_3to1

    # Extract sequence and residues
    sequence = ""
    residue_info = []

    for chain in structure[0]:  # first model
        if chain.id == chain_id:
            for residue in chain:
                resname = residue.get_resname().strip()
                try:
                    one_letter = three_to_one[resname.capitalize()]
                    sequence += one_letter
                    residue_info.append({
                        'residue_name': resname,
                        'residue_id': residue.get_id()[1],
                        'chain_id': chain.id,
                        'one_letter': one_letter
                    })
                except KeyError:
                    continue  # skip non-standard residues

    # Calculate hydropathy
    analysed_seq = ProteinAnalysis(sequence)
    hydro_values = analysed_seq.protein_scale(param_dict=ProtParamData.kd, window=window)
    offset = window // 2

    # Add hydropathy to residue info
    for i in range(len(residue_info)):
        if offset <= i < len(residue_info) - offset:
            residue_info[i]['hydropathy'] = round(hydro_values[i - offset], 3)
        else:
            residue_info[i]['hydropathy'] = None

    # Add index
    for i, r in enumerate(residue_info):
        r['index'] = i + 1

    return pd.DataFrame(residue_info)

#  Run this
df_hydro = calculate_hydropathy_df("./../target/7z14.pdb", chain_id='F', window=9)
print(df_hydro.head())

  residue_name  residue_id chain_id one_letter  hydropathy  index
0          MET           1        F          M         NaN      1
1          ILE           2        F          I         NaN      2
2          CYS           3        F          C         NaN      3
3          TYR           4        F          Y         NaN      4
4          ASN           5        F          N        -0.5      5


In [36]:
# Ensure consistent residue name format
df_hydro = df_hydro[['chain_id', 'residue_id', 'residue_name', 'hydropathy']]
df_hydro['residue_name'] = df_hydro['residue_name'].str.upper()

# Merge only hydropathy into df_target
df_merged = df_target.merge(
    df_hydro,
    on=['chain_id', 'residue_id', 'residue_name'],
    how='left'  # keep all rows from df_target
)

# Get list of columns
cols = df_merged.columns.tolist()
# Find the index of 'accessibility' column
access_idx = cols.index('accessibility')
# Remove 'hydropathy' from its current location
cols.remove('hydropathy')
# Insert 'hydropathy' right after 'accessibility'
cols.insert(access_idx + 1, 'hydropathy')
# Reorder dataframe
df_merged = df_merged[cols]

# df_merged = df_merged.sort_values(by=['accessibility', 'hydropathy'], ascending=[False, False]).reset_index(drop=True)

# Display result
from IPython.display import display, Markdown
display(Markdown(df_merged.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |    psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|-------:|--------:|:---------------|
| F          |            1 | MET            | nonpolar   | loop      | -         |       0.308511  |      nan     |  360   |  128.5 |    1.36 |                |
| F          |            2 | ILE            | nonpolar   | sheet     | E         |       0.301775  |      nan     | -109.3 |  125   |    1.31 |                |
| F          |            3 | CYS            | polar      | sheet     | E         |       0         |      nan     | -150.9 |  162.2 |    1.28 |                |
| F          |            4 | TYR            | polar      | sheet     | E         |       0.166667  |      nan     |  -70.7 |  138.2 |    1.21 |                |
| F          |            5 | ASN            | polar      | loop      | -         |       0.356688  |       -0.5   | -115.1 |   13.9 |    1.19 |                |
| F          |            6 | GLN            | polar      | loop      | -         |       0.0757576 |       -1.1   |  -69.7 |  177.8 |    1.12 |                |
| F          |            7 | GLN            | polar      | loop      | B         |       0.308081  |       -1.778 | -132.8 |  137   |    1.02 |                |
| F          |            8 | SER            | polar      | loop      | T         |       0         |       -2.233 |   60.7 | -131.5 |    1.04 |                |
| F          |            9 | SER            | polar      | loop      | T         |       0.369231  |       -2.167 | -110.5 |   39.9 |    1.06 |                |
| F          |           10 | GLN            | polar      | loop      | P         |       0.181818  |       -1.856 |  -73.4 |  166.8 |    1.07 |                |
| F          |           11 | PRO            | nonpolar   | loop      | P         |       0.647059  |       -1.9   |  -58.9 |  144.5 |    1.13 |                |
| F          |           12 | PRO            | nonpolar   | loop      | P         |       0.669118  |       -1.589 |  -60.5 |  141.8 |    1.16 |                |
| F          |           13 | THR            | polar      | loop      | -         |       0.514085  |       -1.222 | -147.2 |  135.7 |    1.2  |                |
| F          |           14 | THR            | polar      | sheet     | E         |       0.471831  |       -1.222 | -117.4 |  158.9 |    1.24 |                |
| F          |           15 | LYS            | positive   | sheet     | E         |       0.35122   |       -1.222 | -145.6 |  157   |    1.28 |                |
| F          |           16 | THR            | polar      | sheet     | E         |       0.838028  |       -1.122 |  -97.4 |  138   |    1.37 |                |
| F          |           17 | CYS            | polar      | loop      | -         |       0.222222  |       -1.033 | -151.5 |  157.7 |    1.4  |                |
| F          |           18 | SER            | polar      | loop      | S         |       0.892308  |       -0.678 |  -86.3 |  -11.9 |    1.42 |                |
| F          |           19 | GLU            | negative   | loop      | -         |       0.572165  |       -0.744 |  -78.3 |  154   |    1.4  |                |
| F          |           20 | THR            | polar      | loop      | S         |       0.669014  |       -0.744 |  -92.3 |    5.5 |    1.37 |                |
| F          |           21 | SER            | polar      | loop      | -         |       0.130769  |       -1.1   | -125   |  148.9 |    1.34 |                |
| F          |           22 | CYS            | polar      | sheet     | E         |       0         |       -1.456 | -141.6 |  134   |    1.28 |                |
| F          |           23 | TYR            | polar      | sheet     | E         |       0.108108  |       -1.467 | -123   |  153.4 |    1.17 |                |
| F          |           24 | LYS            | positive   | sheet     | E         |       0.243902  |       -1.578 | -119.6 |  122.3 |    1.12 |                |
| F          |           25 | LYS            | positive   | sheet     | E         |       0.0682927 |       -1.889 | -112.6 |  124.1 |    1.07 |                |
| F          |           26 | THR            | polar      | sheet     | E         |       0.21831   |       -2.156 | -127.8 |  132.1 |    1.06 |                |
| F          |           27 | TRP            | nonpolar   | sheet     | E         |       0.105727  |       -2.933 | -156.8 |  163.6 |    1.02 |                |
| F          |           28 | ARG            | positive   | sheet     | E         |       0.362903  |       -2.833 | -119   |  138.9 |    1.02 |                |
| F          |           29 | ASP            | negative   | loop      | -         |       0.0429448 |       -2.478 | -116.9 | -169.3 |    0.98 |                |
| F          |           30 | HIS            | positive   | loop      | S         |       0.157609  |       -1.544 |  -64.2 |  -18.9 |    0.94 |                |
| F          |           31 | ARG            | positive   | loop      | S         |       0.314516  |       -0.967 |  -99.7 |  -56.6 |    0.94 |                |
| F          |           32 | GLY            | nonpolar   | loop      | -         |       0.142857  |       -1.256 | -156.1 | -151.9 |    0.97 |                |
| F          |           33 | THR            | polar      | sheet     | E         |       0.34507   |       -1.256 | -112.8 |  139.3 |    1.01 |                |
| F          |           34 | ILE            | nonpolar   | sheet     | E         |       0.0473373 |       -0.911 | -117.1 |  131.3 |    1.02 |                |
| F          |           35 | ILE            | nonpolar   | sheet     | E         |       0.159763  |       -0.278 | -115   |  125.1 |    1.05 |                |
| F          |           36 | GLU            | negative   | sheet     | E         |       0.298969  |        0.178 | -119.1 |  133.3 |    1.1  |                |
| F          |           37 | ARG            | positive   | sheet     | E         |       0.0887097 |        0.5   | -127.1 |  146.6 |    1.13 |                |
| F          |           38 | GLY            | nonpolar   | sheet     | E         |       0.0595238 |        0.4   |  178.1 | -168.1 |    1.21 |                |
| F          |           39 | CYS            | polar      | sheet     | E         |       0.177778  |       -0.533 |  -94.2 |  163.2 |    1.29 |                |
| F          |           40 | GLY            | nonpolar   | loop      | S         |       0.142857  |       -0.567 |   65.2 | -171.1 |    1.28 |                |
| F          |           41 | CYS            | polar      | loop      | -         |       0.281481  |       -0.611 | -157.3 |   98.2 |    1.27 |                |
| F          |           42 | PRO            | nonpolar   | loop      | -         |       0.588235  |       -0.289 |  -86.6 | -177.9 |    1.23 |                |
| F          |           43 | LYS            | positive   | loop      | -         |       0.97561   |       -0.289 |  -74.4 |  149.3 |    1.2  |                |
| F          |           44 | VAL            | nonpolar   | loop      | -         |       0.352113  |       -0.067 | -130.8 |  159.6 |    1.12 |                |
| F          |           45 | LYS            | positive   | loop      | -         |       0.312195  |       -0.456 |  -68.9 |  155.8 |    1.06 |                |
| F          |           46 | PRO            | nonpolar   | loop      | T         |       0.882353  |       -0.311 |  -60.5 |  143.3 |    1.06 |                |
| F          |           47 | GLY            | nonpolar   | loop      | T         |       0.761905  |       -0.489 |   93.2 |   10.7 |    1.05 |                |
| F          |           48 | ILE            | nonpolar   | loop      | -         |       0.112426  |        0.222 | -119.7 |  130.6 |    1.06 |                |
| F          |           49 | LYS            | positive   | sheet     | E         |       0.731707  |        0.033 |  -91.2 |  141.1 |    1.11 |                |
| F          |           50 | LEU            | nonpolar   | sheet     | E         |       0.335366  |       -0.033 | -136.3 |  153.4 |    1.17 |                |
| F          |           51 | HIS            | positive   | loop      | -         |       0.559783  |        0.067 | -133.8 |   53.2 |    1.25 |                |
| F          |           52 | CYS            | polar      | loop      | -         |       0.555556  |       -0.278 |  -40.9 |  132.5 |    1.31 |                |
| F          |           53 | CYS            | polar      | loop      | B         |       0.177778  |       -1.211 | -147   |  161.1 |    1.32 |                |
| F          |           54 | ARG            | positive   | loop      | -         |       0.741935  |       -0.5   | -121.3 |  -18.5 |    1.32 |                |
| F          |           55 | THR            | polar      | loop      | S         |       0.647887  |       -1.311 |  -96.3 |  175.4 |    1.32 |                |
| F          |           56 | ASP            | negative   | loop      | S         |       0.546012  |       -1.344 |  -81.6 |  132.9 |    1.33 |                |
| F          |           57 | LYS            | positive   | loop      | S         |       0.546341  |      nan     |   55.5 |   63.6 |    1.3  |                |
| F          |           58 | CYS            | polar      | loop      | -         |       0.237037  |      nan     | -123.3 |   -0.5 |    1.31 |                |
| F          |           59 | ASN            | polar      | loop      | -         |       0         |      nan     |  -99   |   34.3 |    1.26 |                |
| F          |           60 | ASN            | polar      | loop      | -         |       0.745223  |      nan     |  -59.9 |  360   |    1.25 |                |

In [37]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.7)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   |   residue_id | residue_name   | property   | ss_type   | ss_code   |   accessibility |   hydropathy |    phi |   psi |   plddt | is_interface   |
|:-----------|-------------:|:---------------|:-----------|:----------|:----------|----------------:|-------------:|-------:|------:|--------:|:---------------|
| F          |           16 | THR            | polar      | sheet     | E         |        0.838028 |       -1.122 |  -97.4 | 138   |    1.37 |                |
| F          |           18 | SER            | polar      | loop      | S         |        0.892308 |       -0.678 |  -86.3 | -11.9 |    1.42 |                |
| F          |           43 | LYS            | positive   | loop      | -         |        0.97561  |       -0.289 |  -74.4 | 149.3 |    1.2  |                |
| F          |           46 | PRO            | nonpolar   | loop      | T         |        0.882353 |       -0.311 |  -60.5 | 143.3 |    1.06 |                |
| F          |           47 | GLY            | nonpolar   | loop      | T         |        0.761905 |       -0.489 |   93.2 |  10.7 |    1.05 |                |
| F          |           49 | LYS            | positive   | sheet     | E         |        0.731707 |        0.033 |  -91.2 | 141.1 |    1.11 |                |
| F          |           54 | ARG            | positive   | loop      | -         |        0.741935 |       -0.5   | -121.3 | -18.5 |    1.32 |                |
| F          |           60 | ASN            | polar      | loop      | -         |        0.745223 |      nan     |  -59.9 | 360   |    1.25 |                |

In [32]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.7) & (df_merged['hydropathy'] > 0.4)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   | residue_id   | residue_name   | property   | ss_type   | ss_code   | accessibility   | hydropathy   | phi   | psi   | plddt   | is_interface   |
|------------|--------------|----------------|------------|-----------|-----------|-----------------|--------------|-------|-------|---------|----------------|

In [33]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.6) & (df_merged['hydropathy'] > 0.3)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   | residue_id   | residue_name   | property   | ss_type   | ss_code   | accessibility   | hydropathy   | phi   | psi   | plddt   | is_interface   |
|------------|--------------|----------------|------------|-----------|-----------|-----------------|--------------|-------|-------|---------|----------------|

In [34]:
df_filtered = df_merged[(df_merged['accessibility'] > 0.5) & (df_merged['hydropathy'] > 0.3)]
display(Markdown(df_filtered.to_markdown(index=False)))

| chain_id   | residue_id   | residue_name   | property   | ss_type   | ss_code   | accessibility   | hydropathy   | phi   | psi   | plddt   | is_interface   |
|------------|--------------|----------------|------------|-----------|-----------|-----------------|--------------|-------|-------|---------|----------------|

## Detecting Inter-Chain Contacts
To determine whether a particular chain (e.g., 'F') makes contact with other chains in a PDB structure, you can check for interatomic distances between residues in chain 'F' and residues in all other chains. If any atoms are closer than a distance threshold (commonly ~5 Å), it's considered a contact.

In [41]:
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

def get_chain_contacts(pdb_file, target_chain_id='F', distance_threshold=3.5):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    model = structure[0]  # Use the first model
    
    # Collect atoms from chain F and other chains
    target_atoms = []
    other_atoms = []
    
    for chain in model:
        for residue in chain:
            if not is_aa(residue):
                continue
            for atom in residue:
                if chain.id == target_chain_id:
                    target_atoms.append(atom)
                else:
                    other_atoms.append(atom)

    # Search for contacts using NeighborSearch
    ns = NeighborSearch(other_atoms)
    contacts = set()

    for atom in target_atoms:
        neighbors = ns.search(atom.coord, distance_threshold)
        for neighbor in neighbors:
            neighbor_chain = neighbor.get_parent().get_parent().id
            if neighbor_chain != target_chain_id:
                contacts.add(neighbor_chain)

    return list(contacts)

pdb_file = "./../target/7z14.pdb"
contacting_chains = get_chain_contacts(pdb_file, target_chain_id='F')
print(f"Chain 'F' contacts: {contacting_chains}")

Chain 'F' contacts: ['C', 'D']


### Residue-Level Contact Map
identify residue-residue contacts between chain 'F' and other chains in a PDB file. This means listing which residue in chain 'F' is close to which residue in other chains, based on a distance cutoff (usually ~5.0 Å).

In [47]:
from Bio.PDB import PDBParser, NeighborSearch, is_aa

def get_chain_residue_contacts(pdb_file, target_chain_id='F', distance_threshold=5.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file)
    model = structure[0]

    target_residues = [res for res in model[target_chain_id] if is_aa(res)]

    # Build list of all atoms (excluding the target chain) and map them to residues
    other_atoms = []
    atom_to_residue = {}
    for chain in model:
        if chain.id == target_chain_id:
            continue
        for residue in chain:
            if not is_aa(residue):
                continue
            for atom in residue:
                other_atoms.append(atom)
                atom_to_residue[atom] = residue

    # Build neighbor search index
    ns = NeighborSearch(other_atoms)

    # Store residue-residue contacts
    contacts = []

    for res_f in target_residues:
        for atom in res_f:
            neighbors = ns.search(atom.coord, distance_threshold)
            for neighbor in neighbors:
                res_other = atom_to_residue[neighbor]
                contact = {
                    'F_residue_name': res_f.get_resname(),
                    'F_residue_id': res_f.get_id()[1],
                    'contact_chain': res_other.get_parent().id,
                    'contact_residue_name': res_other.get_resname(),
                    'contact_residue_id': res_other.get_id()[1]
                }
                contacts.append(contact)

    # Remove duplicates
    contacts = [dict(t) for t in {tuple(d.items()) for d in contacts}]
    return contacts



contacts = get_chain_residue_contacts("./../target/7z14.pdb", target_chain_id='F', distance_threshold=3.5)

import pandas as pd
df_contacts = pd.DataFrame(contacts)
display(df_contacts)

,F_residue_name,F_residue_id,contact_chain,contact_residue_name,contact_residue_id
0,ILE,34,D,TYR,190
1,SER,9,D,PRO,194
2,HIS,30,C,TRP,57
3,ASP,29,D,TYR,190
4,SER,8,D,TYR,189
5,GLN,7,D,TRP,187
6,GLN,10,D,TYR,189
7,SER,8,D,TYR,190
8,SER,8,D,THR,191
9,LYS,45,C,PRO,181


In [48]:
f_residue_string = ",".join(df_contacts["F_residue_id"].apply(lambda x: f"F{x}").drop_duplicates())
print(f_residue_string)

F34,F9,F30,F29,F8,F7,F10,F45,F31,F28,F33,F25,F48
